In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.remote.command import Command
from selenium.webdriver.common.by import By

import socket

import undetected_chromedriver as uc

import time
import pandas as pd
import regex as re


In [2]:
def get_product_one_page(page_source):
    
    list_names = []
    list_area = []
    list_price = []
    list_location = []
    list_url = []
    list_square = []
    list_phone = []
    
    
    item_regex = re.compile('.*AdItem_wrapperAdItem__.*')
    item_wrapper = page_source.find_all('li', {"class" : item_regex})
    i = 0
    for ad_item in item_wrapper:
        # Print out
        i +=1
        print("Proccess: {0}/{1}".format(i, len(item_wrapper)))
        
        # NAME
        ad_item_name = ad_item.find('h3').get_text()
        
        # PRICE
        price_regex = re.compile('.*_adPriceNormal__.*')
        ad_item_price = ad_item.find('p', {"class": price_regex}).get_text()
        
        area_regex = re.compile('.*AdItemFooter_item.*')
        
        # AREA
        ad_item_area = "N/A"
        try:
            ad_item_area = ad_item.find_all('span', {"class": area_regex})[-1].get_text().replace('-', '')
        except:
            print('Can not get area')
            
        ad_item_path = ad_item.find('a')['href']
        ad_item_url = "https://www.nhatot.com{0}".format(ad_item_path)
        
        # Item details
        item_page_source = get_page_source(ad_item_url, True)
        
        # LOCATION
        location_regex = re.compile('.*AdParam_address__.*')
        ad_item_location = 'N/A'
        try:
            ad_item_location = item_page_source.find('div', {"class": location_regex}).get_text().replace('Xem bản đồ', '')
        except:
            print('Can not get Location')
        
        # SQUARE
        square_regex = re.compile('.*AdDecriptionVeh_squareMetre__.*')
        ad_item_square = "N/A"
        try:
            ad_item_square = item_page_source.find('span', {"class": square_regex}).get_text()
        except:
            print('Can not get Square')
            
        # PHONE
        phone_regex = re.compile('.*ShowPhoneButton_phoneClicked.*')
        ad_item_phone = "N/A"
        try:
            ad_item_phone = item_page_source.find('div', {"class": phone_regex}).get_text()
        except:
            print('Can not get Phone')
            
            
            
            
        list_names.append(ad_item_name)
        list_area.append(ad_item_area)
        list_price.append(ad_item_price)
        list_location.append(ad_item_location)
        list_url.append(ad_item_url)
        list_square.append(ad_item_square)
        list_phone.append(ad_item_phone)
        
        time.sleep(1)
            
            
    df = pd.DataFrame()
    df['title'] = list_names
    df['area'] = list_area
    df['price_VND'] = list_price
    df['square'] = list_square
    df['location'] = list_location
    df['url'] = list_url
    df['phone'] = list_phone
        
    return df

In [4]:
options = webdriver.ChromeOptions() 
driver = uc.Chrome(options=options)
timeout = 5
driver.set_page_load_timeout(timeout)

def get_page_source(url, get_phone=False ,load_fast=False):
    if load_fast == True:
        options.page_load_strategy = 'eager'
    else:
        options.page_load_strategy = 'normal'
        
    page_source = ""
    try:
        driver.get(url)
    except TimeoutException:
        print("Timed out waiting for page to load")
    
    if get_phone:
        try:
            show_number_button = WebDriverWait(driver, 1).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div[class*='ShowPhoneButton_phoneButton']")))
            driver.execute_script("arguments[0].click()", show_number_button[0])
        except Exception as e:
            print('Can not get number')
            print('Failed to upload to ftp: %s', repr(e))
        
    try:
        time.sleep(0.5)
        page_source = driver.page_source
    except:
        print("No page_source - retry")
        try:
            driver.close()
            driver.get(url)
            page_source = driver.page_source
        except:
            print("No page_source - end")
            
    
    return BeautifulSoup(page_source,"html.parser")

In [5]:
df_list = []


In [6]:
for i in range(1,9):
    print("Page {0}".format(i))
    page_url = 'https://www.nhatot.com/tags/phong-tro-co-gac-lung-binh-thanh?price=*-4200000&page={0}'.format(i)
    page_source = get_page_source(page_url)
    df_page = get_product_one_page(page_source)
    df_list.append(df_page)

Page 1
Proccess: 1/22
Proccess: 2/22
Proccess: 3/22
Proccess: 4/22
Proccess: 5/22
Proccess: 6/22
Proccess: 7/22
Proccess: 8/22
Can not get area
Proccess: 9/22
Can not get area
Proccess: 10/22
Can not get area
Proccess: 11/22
Can not get area
Proccess: 12/22
Can not get area
Proccess: 13/22
Can not get area
Proccess: 14/22
Can not get area
Proccess: 15/22
Can not get area
Proccess: 16/22
Can not get area
Proccess: 17/22
Can not get area
Proccess: 18/22
Can not get area
Proccess: 19/22
Can not get area
Proccess: 20/22
Can not get area
Proccess: 21/22
Can not get area
Proccess: 22/22
Can not get area
Page 2
Proccess: 1/20
Proccess: 2/20
Proccess: 3/20
Proccess: 4/20
Proccess: 5/20
Proccess: 6/20
Proccess: 7/20
Proccess: 8/20
Can not get area
Proccess: 9/20
Can not get area
Proccess: 10/20
Can not get area
Proccess: 11/20
Can not get area
Proccess: 12/20
Can not get area
Proccess: 13/20
Can not get area
Proccess: 14/20
Can not get area
Proccess: 15/20
Can not get area
Proccess: 16/20
Can n

In [25]:
print(df_list[0])

                                                title       area  \
0   Phòng trọ Bạch Đằng 18m2 sạch đẹp, an ninh,giá...  Phường 24   
1   SIÊU HOT PHÒNG CÓ GÁC KHÉP KÍN - NGAY HÀNG XAN...  Phường 21   
2   Nhà trọ full nội thất Hàng Xanh, Điện Biên Phủ...  Phường 17   
3   PHÒNG CHO THUÊ FULL NỘI THẤT GIÁ SINH VIÊN NGU...   Phường 5   
4      Phòng biệt thự Bình Lợi, full nội thất, có gác  Phường 13   
5            Phòng trọ có  tủ lạnh không nội thất.13m  Phường 11   
6        Phòng Trọ 12m2, Ung Văn Khiêm, Q.Bình Thạnh.  Phường 25   
7                     Phòng trọ đẹp giá rẻ Bình Thạnh        N/A   
8   Trọ rẻ nhiều mức giá 1tr2-4tr, Trần Bình Trọng...        N/A   
9   SIÊU HOT PHÒNG CÓ GÁC KHÉP KÍN - NGAY HÀNG XAN...        N/A   
10  Nhà trọ full nội thất Hàng Xanh, Điện Biên Phủ...        N/A   
11  Hẻm 136 Vạn Kiếp, 18m2 có máy lạnh, gác lửng, ...        N/A   
12                              Phòng trọ 15 m có gác        N/A   
13    Phòng cho thuê 477 Xô Viết Nghệ Tĩnh p26 B

In [7]:
final_df = pd.concat(df_list )
final_df

,title,area,price_VND,square,location,url,phone
0,"Phòng 3.7t- 4.7t hàng xanh,( giảm 200k/ phòng",Phường 25,4 triệu/tháng,- 30 m2,"356/20G, Xô Viết Nghệ Tĩnh, Phường 25, Quận Bì...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0908094289
1,"Phòng biệt thự Bình Lợi, full nội thất, có gác",Phường 13,"3,6 triệu/tháng",- 20 m2,"106/45/4, Bình Lợi, Phường 13, Quận Bình Thạnh...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0933333033
2,phòng mới xây cho thuê,Phường 25,4 triệu/tháng,- 18 m2,"428, Đường Xô Viết Nghệ Tĩnh, Phường 25, Quận ...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0378445445
3,Phòng trọ đẹp giá rẻ Bình Thạnh,Phường 25,"1,9 triệu/tháng",- 10 m2,"Ung Văn Khiêm, Phường 25, Quận Bình Thạnh, Tp ...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0772988334
4,"Phòng biệt thự Bình Lợi, full nội thất, có gác",Phường 13,"3,6 triệu/tháng",- 20 m2,"106/45/4, Bình Lợi, Phường 13, Quận Bình Thạnh...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0933333033
...,...,...,...,...,...,...,...
15,PHÒNG TRỌ CHÍNH CHỦ CHO THUÊ,N/A,3 triệu/tháng,- 20 m2,"trần bình trọng, Phường 5, Quận Bình Thạnh, Tp...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0346433435
16,CHO THUÊ PHÒNG TRỌ MẶT TIỀN GIÁ RẺ,N/A,2 triệu/tháng,- 20 m2,"Quốc lộ 13, Phường 26, Quận Bình Thạnh, Tp Hồ ...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0359502262
17,"PASS phòng trọ 20m2 hẻm 82 Nguyễn Xí, Bình Thạnh",N/A,"3,8 triệu/tháng",- 20 m2,"Hẻm 82, Phường 26, Quận Bình Thạnh, Tp Hồ Chí ...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0799739248
18,phòng trọ giá rẻ,N/A,"3,5 triệu/tháng",- 16 m2,"Nguyễn Xí, Phường 26, Quận Bình Thạnh, Tp Hồ C...",https://www.nhatot.com/thue-phong-tro-quan-bin...,0935335938


In [8]:
# Save csv
final_df.to_csv('final_df_bthanh_less_4tr2_v2.csv')

In [14]:
for i in range(1,5):
    print(i)

1
2
3
4


In [4]:
options = webdriver.ChromeOptions() 
driver = uc.Chrome(options=options)
timeout = 5
driver.set_page_load_timeout(timeout)
url = 'https://www.nhatot.com/thue-phong-tro-thanh-pho-thu-duc-tp-ho-chi-minh/114360256.htm#px=SR-stickyad-[PO-1][PL-top]'
driver.get(url)


In [5]:
show_number_button = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div[class*='ShowPhoneButton_phoneButton']")))
# show_number_button = driver.find_element(By.CSS_SELECTOR, "div[class*='ShowPhoneButton_phoneButton']")
print(show_number_button)

[<undetected_chromedriver.webelement.WebElement (session="32d74d91481dc991a94f41f2f28b41f5", element="f.A6966FCD6B3AB335ADC6993D3D02AC93.d.DB1300E4E35E415F2ABF4BC75D2B375E.e.279")>]


In [9]:
driver.execute_script("arguments[0].click()", show_number_button[0])

In [6]:
show_number_button[0].click()

ElementClickInterceptedException: Message: element click intercepted: Element <div class="ShowPhoneButton_phoneButton__p5Cvt ShowPhoneButton_phoneNotClicked__dlQn_">...</div> is not clickable at point (1175, 697). Other element would receive the click: <div class="aw__s13upwxx"></div>
  (Session info: chrome=122.0.6261.69)
Stacktrace:
0   undetected_chromedriver             0x00000001033d12a8 undetected_chromedriver + 4625064
1   undetected_chromedriver             0x00000001033c8e53 undetected_chromedriver + 4591187
2   undetected_chromedriver             0x0000000102fc7e6a undetected_chromedriver + 392810
3   undetected_chromedriver             0x000000010301a113 undetected_chromedriver + 729363
4   undetected_chromedriver             0x0000000103017fe1 undetected_chromedriver + 720865
5   undetected_chromedriver             0x0000000103015a17 undetected_chromedriver + 711191
6   undetected_chromedriver             0x0000000103014de2 undetected_chromedriver + 708066
7   undetected_ch